In [1]:
import os
import pandas as pd
import json

In [2]:
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

c:\Users\kimso\anaconda3\envs\test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 데이터 불러오기

#### 1. 가사

In [3]:
lyrics1 = pd.read_csv("./dataset/label_result_song_short.csv")
lyrics2 = pd.read_csv("./dataset/translated_lyrics.csv")

lyrics = lyrics2.merge(lyrics1, on=["index", "id", "title", "singer", "genres", "lyrics"])

In [4]:
lyrics.head()

,index,id,title,singer,genres,lyrics,translated_lyrics,sentiment,sentiment_label
0,1,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,Ay ay ay ay ay,Ay ay ay ay ay,2,분노
1,2,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,거울 속에 내 표정 봐 봐,거울 속에 내 표정 봐 봐,5,당황
2,3,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,느낌 So good 기다려온 D-day,느낌 그래서 좋아요. 기다려온 D-하루,0,기쁨
3,4,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,연습했던 손든 인사도 그대로 하면 돼,연습했던 손든 인사도 그대로 하면 돼,0,기쁨
4,5,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,Hairstyle check하고 한 번 Turn around,머리스타일 확인하고 한 번 회전 주변,2,분노


#### 2. 소설

In [5]:
novel_final = pd.read_csv('./dataset/classified_novel1.csv')
data = novel_final.copy()

In [ ]:
# data = data.sample(n=50000, random_state=42).reset_index(drop=True)

len(data)

50000

In [7]:
data.head()

,Column1,file_id,title,paragraph_id,text,labels,sentiment
0,347937,WARW1900000524,남도빨치산 6,WARW1900000524.1.2310,"""손들고 나왓!""",0.0,기쁨
1,570921,WARW1900000808,제갈공명 2-하늘로 치솟아 뜻을 펼치다,WARW1900000808.1.1898,"""그 점은 이 양이 생각해둔 바 있으니 과히 염려하지 마십시오. 틀림없이 연환계를 ...",0.0,기쁨
2,257278,WARW1900000461,890만번 주사위 던지기,WARW1900000461.1.504,"""102…103…104…….""",4.0,불안
3,702456,WARW1900001046,경찰청장 박전전 3,WARW1900001046.1.841,그 두 가지 사실이 무엇을 의미하는가?,1.0,슬픔
4,228104,WARW1900000425,퇴원,WARW1900000425.1.1765,"""어서 그냥 쇠새끼나 쫓아가보거라. 꼴짐은 내가 거둬 지고 가마.""",2.0,분노


### Preprocessing

In [8]:
print("NaN 개수:", data['text'].isna().sum())  # NaN 개수 확인
print("None 포함 여부:", data['text'].isnull().sum())  # None 개수 확인

NaN 개수: 6
None 포함 여부: 6


In [9]:
data['text'] = data['text'].fillna("")

In [10]:
print("NaN 개수:", data['text'].isna().sum())  # NaN 개수 확인
print("None 포함 여부:", data['text'].isnull().sum())  # None 개수 확인

NaN 개수: 0
None 포함 여부: 0


In [11]:
# 같은 title을 가진 데이터에서 100개씩 묶어 그룹화
data["group"] = data.groupby("title").cumcount() // 100  # 100개 단위 그룹 생성

In [12]:
data = data.groupby(["file_id", "group"]).agg({
    "title": "first",  # 대표 file_id
    "text": list,  # 100개씩 묶음
}).reset_index()

# 불필요한 sentiment 칼럼 삭제
data = data.drop(columns=["group", "file_id"])

data.head()

,title,text
0,이야기꾼 구연설화,"[“당신 싻은 많이 줄 테니까, 좀 멀리 어따 숭겨 노라구 말여. 배밑이 어디 다른..."
1,이야기꾼 구연설화,"[그랬어., 자기는 모르는 일이지. 아 이거 꿈같은 얘기 아녀?, “왜 울어?”, ..."
2,이야기꾼 구연설화,"[‘…? 있이까 모르지이.’, 밥얼 싸다 놓구 나무를 비구 앉었, 벼서 끊어. 지금..."
3,이야기꾼 구연설화,"[아 그라더니 뜻밖에 또 뭐라구능 게 아니라,, 아이고오! 흐흐. 그런 으뭉한 눔이..."
4,이야기꾼 구연설화,"[“제자먼 이 마룽이 와 울래라.”, 쌀 닷 말 박기루 하구 경을 읽어. 허허… 저..."


In [13]:
data['text'] = data['text'].apply(lambda x: ' '.join(map(str, x)) if isinstance(x, list) else str(x))

# 불필요한 문자 제거
data['text'] = data['text'].str.replace(r"[\[\],']", "", regex=True)

In [14]:
data.head()

,title,text
0,이야기꾼 구연설화,“당신 싻은 많이 줄 테니까 좀 멀리 어따 숭겨 노라구 말여. 배밑이 어디 다른 디...
1,이야기꾼 구연설화,그랬어. 자기는 모르는 일이지. 아 이거 꿈같은 얘기 아녀? “왜 울어?” 아리랑 ...
2,이야기꾼 구연설화,‘…? 있이까 모르지이.’ 밥얼 싸다 놓구 나무를 비구 앉었 벼서 끊어. 지금. “...
3,이야기꾼 구연설화,아 그라더니 뜻밖에 또 뭐라구능 게 아니라 아이고오! 흐흐. 그런 으뭉한 눔이 있어...
4,이야기꾼 구연설화,“제자먼 이 마룽이 와 울래라.” 쌀 닷 말 박기루 하구 경을 읽어. 허허… 저녁이...


In [15]:
data.loc[0]["text"]

'“당신 싻은 많이 줄 테니까 좀 멀리 어따 숭겨 노라구 말여. 배밑이 어디 다른 디루 가게 모래밭이다 숭겨 노라.”구. 살구 있을 때에 인제 문왕 살어서. 문왕을 살러 가는 챔인디. 그 자기 본마느래. 그 밥 읃어다 주던 마느래가 시집갔으야 가난해서 못 살어. 그래 그 뭐 태공이가 문왕으루 간다아 응 항깨 왕으루 강깨 따러갈라구 나왔어. 와서 절을 함성(하면서) “돌아오는 글자가 있냐.”구. 갔는디. “그래 월매나 남었능가?” “개구리두 모가지가 질어 소리가 크냐?” 그래 삼척을 가능 거여 지금. 그래 삼척 그 워느 한 저어 삼척 거가 어디냐 뭐 삼수갑산 하는 딘디 상골짝이랴. 그래 생전 그런 디 앙 가본 큰애기가 그런 디를 가는디 월매나 다리가 아푸구 다리 아풀 거여. 산을 넘구 넘어서 거 가닝깨 아 어머니가 “하이구 워트게 성님 죽여?” 그래 워디서 손님이 왔어. 참 그렁깨 말여? 그래서 하늘에서 옥황상제가 그랬능가 누가 그랬능가 동아줄을 넬콰 줘서 지집아 머슴아 올라갔어 인제. 하늘로. “아 아 여보쇼. 담배 좀 한 대 주쇼. 아 담배는 있지만 추져서(축축해서) 몹 먹겄소.” 짓넌디. 그때 마침 저어 대왱이 거기를 지내는디. 그라구 집을 지커덩. 그래 물었어. 저기 저기 저기 봐요. 덱우산(덕유산) 중이 익고. “아이고 여보! 워트갸? 깨졌네.” 그러니. 집이두 못 와 인제. 돈이 있이야 오지. 가서 봉깨 참 뭐이 이케 봉깨 붉우수름한 물이 가뜩 괬더랴. 그래 이케 눌룽깨 쭉 깔릴 거 아녀? “왜 웂어? 많지.” “아이구 우리 집은 이릏게 흠해요. 흠햐. 앉으라.”구. 웃으며 그래 인제 그 노인덜이 노인덜이 인제 갔어. 거기를. 가서 봉깨 사람언 욱구 워서 아 우는 소리가 나. 가만히 봉깨 그 두루매기 속이서 울어. 어린애가 깟난 어른애가 돼 가지구 한 말하자먼 돐 돐 돐 돐두 안 돌아갈 만치 컸더랴. 그래 한 적이는 가마안히 바러봉깨 가매다가 말여 큰애기를 하나 실쿠 간다 이거여. 큰애긴디. 각신디. 그래 그 ‘별일이다’ 하구 참 각시… 그렁깨 어떤 

In [16]:
# 데이터셋 클래스 정의
class NovelsDataset(Dataset):
    def __init__(self, data, tokenizer, text_column="input_text", max_length=128):
        self.data = data.reset_index(drop=True)  # 인덱스 리셋 (중복 방지)
        self.tokenizer = tokenizer
        self.text_column = text_column
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.loc[idx, self.text_column] # input_text 가져오기

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)

        # labels 생성 (input_ids 복사 및 패딩 토큰을 -100으로 설정)
        labels = input_ids.clone()
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels,
        }

### Modeling

In [17]:
# KOGPT2 모델과 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

# 모델 초기화
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.bos_token_id = tokenizer.bos_token_id

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [18]:
dataset = NovelsDataset(data, tokenizer, text_column="text")
dataloader = DataLoader(dataset, batch_size=2)

In [19]:
# 학습 인자 설정
training_args = TrainingArguments(
    output_dir='./results/baseline',
    num_train_epochs=5,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=5e-5,
    save_steps=200,
    save_total_limit=2,
    logging_steps=50,
    eval_strategy="epoch",
)

In [20]:
train_novels, eval_novels = train_test_split(data, test_size=0.1, random_state=42)

train_dataset = NovelsDataset(train_novels, tokenizer, text_column="text")
eval_dataset = NovelsDataset(eval_novels, tokenizer, text_column="text")

In [21]:
len(train_novels)

631

In [22]:
# Trainer에 데이터셋 전달
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # 검증 데이터셋
)

In [23]:
sample = train_dataset[1]
print("Input IDs:", sample['input_ids'])
print("Attention Mask:", sample['attention_mask'])
print("Labels:", sample['labels'])

Input IDs: tensor([ 9022,  6855, 11928,  6889, 10203,  9673,   389,  9036, 21538,  8022,
        11273,  7235, 29081, 11781,  8327,  8146,  7788, 25415,  7489,  9230,
         8423,  9856,  8148,  9022, 19841, 24473, 20284, 10341,  9566, 22882,
          389, 36334,  9351,  9138,  9099, 18902,   389,  9089,  7282, 15933,
         7634,  8146, 10489,  7480,  9016,  9376,  9908,  9277, 10724,  9109,
         7162, 10724,  9094, 17302, 15933, 21526, 32664, 11312, 11489,  9414,
        21852, 12354, 20805, 39587,  9432,  7285, 26156, 19912, 40052, 15212,
         9749,  8146, 12502, 15234,  6958, 10348,  9036, 12499,  9192,  7285,
         8277, 45330,  9016, 11275, 26753,  9036, 25940, 13673,  7182, 11213,
         7162, 25265, 21561, 31895,  9368, 21833, 19958, 47198, 30415, 36858,
        28300, 28786, 11057,  7258,  7801,  8094,  9016, 47528,  9037,  6825,
         6825,  8148, 39955,  7501,  9224, 10446, 18412,  9055,  7424, 10480,
        27878,  7003,  8137, 13413, 10310,  9031,  90

In [24]:
# 모델 학습
trainer.train()

c:\Users\kimso\anaconda3\envs\test\lib\site-packages\transformers\integrations\sdpa_attention.py:53: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,No log,4.540387


KeyboardInterrupt: 

In [31]:
device = torch.device("cuda")

In [33]:

# 이야기 생성 함수
def generate_story(lyrics_input):
    # 입력 토큰화
    input_ids = tokenizer.encode(lyrics_input, return_tensors='pt').to(device)

    # 출력 생성
    output = model.generate(
        input_ids, 
        max_new_tokens=150, # 새로 생성할 토큰의 개수를 제한
        num_return_sequences=1, 
        temperature=0.8, 
        top_k=50, 
        top_p=0.9, 
        repetition_penalty=1.2, 
        do_sample=True  # 샘플링 활성화
    )

    # 입력 길이 추적
    input_length = input_ids.shape[1]

    # 생성된 토큰 중 입력 토큰 이후의 부분만 디코딩
    generated_tokens = output[0][input_length:]
    generated_story = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return generated_story

In [24]:
lyrics1 = pd.read_csv("./dataset/label_result_song_short.csv")
lyrics2 = pd.read_csv("./dataset/translated_lyrics.csv")

lyrics = lyrics2.merge(lyrics1, on=["index", "id", "title", "singer", "genres", "lyrics"])

In [25]:
# groupby와 agg를 활용해 변환
lyrics_input = lyrics.groupby("id").agg({
    "title": "first",   # 첫 번째 값 유지
    "singer": "first",  # 첫 번째 값 유지
    "genres": "first",  # 첫 번째 값 유지
    "lyrics": list,  # 첫 번째 값 유지
    "translated_lyrics": list,  # 리스트로 묶음
}).reset_index()

In [26]:
lyrics_input.head()

,id,title,singer,genres,lyrics,translated_lyrics
0,418168,희재,성시경,"발라드, 국내영화","[햇살은 우릴 위해 내리고 , 바람도 서롤 감싸게 했죠 , 우리 웃음속에, 계절은 ...","[햇살은 우릴 위해 내리고 , 바람도 서롤 감싸게 했죠 , 우리 웃음속에, 계절은 ..."
1,418598,친구라도 될 걸 그랬어,거미 (GUMMY),R&B/Soul,"[벌써 넌 내가 편하니, 웃으며 인사 할 만큼, 까맣게 나를 잊었니, 네 곁에 있는...","[벌써 넌 내가 편하니, 웃으며 인사 할 만큼, 까맣게 나를 잊었니, 네 곁에 있는..."
2,711626,살다가,SG 워너비,발라드,"[살아도 사는 게 아니래, 너 없는 하늘에, 창 없는 감옥 같아서, 웃어도 웃는 게...","[살아도 사는 게 아니래, 너 없는 하늘에, 창 없는 감옥 같아서, 웃어도 웃는 게..."
3,1500196,내사람,SG 워너비,R&B/Soul,"[내 가슴속에 사는 사람 내가 그토록 아끼는 사람 , 너무 소중해 마음껏 안아보지도...","[내 가슴속에 사는 사람 내가 그토록 아끼는 사람 , 너무 소중해 마음껏 안아보지도..."
4,1854856,라라라,SG 워너비,발라드,"[그대는 참 아름다워요, 밤 하늘의 별빛보다 빛나요, 지친 나의 마음을 따뜻하게 감...","[그대는 참 아름다워요, 밤 하늘의 별빛보다 빛나요, 지친 나의 마음을 따뜻하게 감..."


In [105]:
i = 725
print(lyrics_input["title"][i])

밤양갱


In [104]:
prompt = " ".join(lyrics_input["translated_lyrics"][i])

print(prompt)

내가 누울 자린 아마도 한참 더 위로 아니 적당히 미끈한 곳에 뿌리내리긴 싫어 내 뒤로 착착 따라붙어 다 예쁘게 줄지어 난 기어코 하늘에 필래 음, 뭐죠? 아 아주 작습니다. le아der 아슬아슬히 나는 홀씨 하나 또 다른 길을 향해서 비행 높은 옆으로 꽃 피우기 혹시 나의 안부를 묻는 누군가 있거든 전해줘 걔는 홀씨가 됐다구 날 따라, 언젠가 밖으로 나가 옆으로 승리 날 따라, 날아가 꼭대기루 여러분 말하자면 '후‘ 저는 어쩌면 파리 여러분 말하자면 '후‘ 그러면 저는 파리 날 따라, 심지어 무언가를 하지 않고 날개 날 따라, 떠올라 공중으루 여러분 말하자면 '후‘ 저는 어쩌면 파리 여러분 말하자면 '후' 그러면 저는 파리 다 날 볼 수 있게 날아 줄게 한가운데 시력을 위해 꼭 지참해 니 햇빛 올려보면 눈부셔 고소공포 하나도 안 무셔 따가운 태양과 무지 가까운 거리 까지 올라가 난 무심히 내려보리 구름을 골라타 간만에 한바탕 싹 어질러볼까 빙글빙그르 나는 홀씨 하나 가파른 바람을 타고 비행 높은 옆으로 꽃 피우기 혹시 나의 안부를 묻는 누군가 있거든 전해줘 걔는 홀씨가 됐다구 날 따라, 언젠가 밖으로 나가 옆으로 승리 날 따라, 날아가 꼭대기루 여러분 말하자면 '후‘ 저는 어쩌면 파리 여러분 말하자면 '후‘ 그러면 저는 파리 날 따라, 심지어 무언가를 하지 않고 날개 날 따라, 떠올라 공중으루 여러분 말하자면 '후‘ 저는 어쩌면 파리 여러분 말하자면 '후' 그러면 저는 파리 앞길이 만만치 않아도 엄살은 뒤로 내 선택이야 늘 그랬듯이 쉬울 확률은 0 남은 거 탈탈 털어줄게 모두 행운을 빌어 구태여 인사하고 갈래 5월 신 무언가를 해야 합니다. 바이러스 야 보세요. 야


In [81]:
# 가사로 이야기 생성
generated_story = generate_story(prompt)
print("생성된 이야기:", generated_story)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# 가사로 이야기 생성
# lyrics_input = ""
# generated_story = generate_story(lyrics_input, tag1="2", tag2="2")
# print("생성된 이야기:", generated_story)